In [1]:
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv



In [2]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(list(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [3]:
HG = load_ABCDH_from_file("datasets/results_300_he.txt")



In [4]:
hL = hl.hLouvain(HG,hmod_type=hmod.linear, 
                                    delta_it = 0.0001, 
                                    delta_phase = 0.0001, 
                                    random_seed = 5673) 

In [5]:
alphas = []
d = 0
c = 0.3
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i)*(1-d))
    
print("alphas", alphas)

#alphas = [1]
    
A, q2, alphas_out = hL.h_louvain_community(alphas = alphas, 
                                                change_mode="communities", change_frequency=c)
qH = hL.combined_modularity(A, hmod.linear, 1)

print("alphas_out", alphas_out)

print("partition",A)
print("gH =",qH)

alphas [0.0, 0.8, 0.96, 0.992, 0.9984, 0.99968, 0.999936, 0.9999872, 0.99999744, 0.999999488, 0.9999998976, 0.99999997952, 0.999999995904, 0.9999999991808, 0.99999999983616, 0.999999999967232, 0.9999999999934464, 0.9999999999986893, 0.9999999999997379, 0.9999999999999476, 0.9999999999999896, 0.9999999999999979, 0.9999999999999996, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
alphas_out [0.0, 0.8, 0.96, 1]
partition [{'93', '50', '238', '43', '57', '213', '149', '190', '211', '284', '66', '74', '297', '169', '179', '103', '276', '241', '80'}, {'86', '23', '157', '132', '115', '257', '267', '42', '184', '279', '107', '214', '178', '138', '289', '146', '30', '98', '106', '208', '273', '164'}, {'1', '17', '25', '242', '248', '181', '282', '150', '11', '54', '218', '194', '153'}, {'139', '183', '219', '215', '278', '260', '295', '39', '52', '232', '281', '239', '8', '76', '91', '160', '5'}, {'95', '99', '36', '59', '296', '222', '294', '163', '47', '142', '69', '10', '170'}, {'111', '1

In [6]:
alphas = []
d = 0
c = 0.3
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i)*(1-d))
    
print("alphas", alphas)

#alphas = [1]
    
Als, A, alphas_out = hL.h_louvain_community_plus_last_step(alphas = alphas, 
                                                change_mode="communities", change_frequency=c)
qH = hL.combined_modularity(A, hmod.linear, 1)

qH1 = hL.combined_modularity(Als, hmod.linear, 1)

print("alphas_out", alphas_out)

print("partition",A)
print("gH =",qH)

print("partition",Als)
print("gH1 =",qH1)

alphas [0.0, 0.8, 0.96, 0.992, 0.9984, 0.99968, 0.999936, 0.9999872, 0.99999744, 0.999999488, 0.9999998976, 0.99999997952, 0.999999995904, 0.9999999991808, 0.99999999983616, 0.999999999967232, 0.9999999999934464, 0.9999999999986893, 0.9999999999997379, 0.9999999999999476, 0.9999999999999896, 0.9999999999999979, 0.9999999999999996, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
alphas_out [0.0, 0.8, 0.96, 1]
partition [{'93', '50', '238', '43', '57', '213', '149', '190', '211', '284', '66', '74', '297', '169', '179', '103', '276', '241', '80'}, {'86', '23', '157', '132', '115', '257', '267', '42', '184', '279', '107', '214', '178', '138', '289', '146', '30', '98', '106', '208', '273', '164'}, {'1', '17', '25', '242', '248', '181', '282', '150', '11', '54', '218', '194', '153'}, {'139', '183', '219', '215', '278', '260', '295', '39', '52', '232', '281', '239', '8', '76', '91', '160', '5'}, {'95', '99', '36', '59', '296', '222', '294', '163', '47', '142', '69', '10', '170'}, {'111', '1

In [7]:
qH = hL.combined_modularity(A, hmod.linear, 1)
print(qH)
two_section_modularity = hL.combined_modularity(A, hmod.linear, 0)
print(two_section_modularity)

0.5251089032577477
0.45126291364965176


In [8]:
with open("Data/results_300_assign.txt", 'r') as file:
    gt = [int(line) for line in file]
A_gt = [x for x in hmod.dict2part({str(i+1):gt[i] for i in range(len(gt))}) if len(x)>0]
gt_linear = hL.combined_modularity(A_gt, hmod.linear, 1)

print("ground-truth partition")
print(A_gt)
print("qh-gt-linear:",gt_linear)

def getAMI_ARI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ari = [d[str(i+1)] for i in range(len(HG.nodes))]
    return ARI(gt, A4ari), AMI(gt, A4ari)

ground-truth partition
[{'77', '183', '144', '260', '96', '112', '65', '134', '81', '286', '254', '203', '14', '117', '79', '272', '285', '227', '300', '70', '124', '197', '271', '191', '148', '135', '182', '125'}, {'268', '68', '71', '280', '110', '34', '83', '121', '123', '89', '90', '141', '154', '92', '230', '119', '28', '209', '185', '198', '189', '41', '16', '226', '87'}, {'247', '228', '222', '78', '12', '116', '95', '221', '173', '18', '288', '46', '143', '88', '296', '229', '251', '72', '223', '109', '210', '151', '152', '266', '265'}, {'32', '82', '235', '4', '114', '270', '162', '167', '236', '44', '261', '283', '186', '180', '13', '255', '290', '62', '147', '177', '40', '63'}, {'49', '2', '200', '104', '35', '216', '31', '140', '257', '174', '58', '269', '195', '166', '102', '258', '67', '29', '299', '201', '45', '172'}, {'93', '50', '238', '43', '57', '213', '149', '190', '211', '284', '66', '74', '297', '169', '179', '103', '241', '276', '80'}, {'146', '30', '279', '86', 

In [9]:
ari, ami = getAMI_ARI(HG,gt,A)
print("ARI =", ari)
print("AMI =", ami)
print("comm =", len(A))
print("comm-gt =", len(A_gt))

ARI = 0.7497036489128945
AMI = 0.8277281509018632
comm = 16
comm-gt = 18


In [10]:
ari, ami = getAMI_ARI(HG,gt,Als)
print("ARI =", ari)
print("AMI =", ami)
print("comm =", len(Als))
print("comm-gt =", len(A_gt))

ARI = 0.7732631300607308
AMI = 0.849292597501855
comm = 16
comm-gt = 18
